In [ ]:
# TAKES AUTH_ AS AN INPUT!!!
import numpy as np
import os
import json
from moleculekit.tools.atomtyper import metal_atypes
from moleculekit.molecule import Molecule
from moleculekit.tools.voxeldescriptors import getVoxelDescriptors
from moleculekit.tools.atomtyper import prepareProteinForAtomtyping
import deeplife_utils
from htmd.ui import *



In [ ]:
# First let's read our two favourite protein and ligand structures
prot = Molecule( '3PTB')

prot.filter('protein')

prot = prepareProteinForAtomtyping(prot)
prot_vox, prot_centers, prot_N = getVoxelDescriptors(prot, buffer=8)

In [ ]:
prot = Molecule('5SXR', validateElements=False)

# proof that it takes auth_ version as an input (here auth_seq_id)
print(prot.get('resid', sel='chain A and protein'))

prot = Molecule('1S00', validateElements=False)

# proof that it takes auth_ version as an input (here auth_asym_id)
prot.filter(f'protein')
print(prot.get('chain', sel='protein'))


In [ ]:
prot = Molecule('3wa2', validateElements=False)
chain_id = 'X'
prot.filter(f'chain {chain_id} and protein')
print(len(set(prot.get('resid', sel=f'chain {chain_id} and protein'))))

sequence = ""
previous_id = float('-inf')
for id, residue in zip(prot.get('resid', sel=f'chain {chain_id} and protein'), prot.get('resname', sel=f'chain {chain_id} and protein')):
    if previous_id == id:
        continue
    else:
        sequence += deeplife_utils.three_to_one(residue)
    previous_id = id
print(sequence)
print(len(sequence))

### GENERATE VOXELS FOR HOLO STRUCTURES
I am not picking every holo structure - only those, whose pockets are less than 75% similar with the other pockets in the scope of current APO structure.

In [ ]:
def generate_voxels(pdb_id, chain_id, pocket, voxel_features_path, voxel_annotations_path):
    binding_residues = [residue.split('_')[1] for residue in pocket]
    binding_residues_query = f'resid {" or resid ".join(binding_residues)}'
    
    try:
        prot = Molecule(pdb_id, validateElements=False)
        prot.filter(f'chain {chain_id} and protein')
        prot = prepareProteinForAtomtyping(prot)
        prot.filter(f"protein or water or element {' '.join(metal_atypes)}")

        prot_voxels, _, N = getVoxelDescriptors(prot, buffer=8)
        annotation_voxels, _, _ = getVoxelDescriptors(prot, buffer=8, userchannels=prot.atomselect(binding_residues_query).reshape(-1,1))
    except:
        return False
            
    features = prot_voxels.reshape(N[0], N[1], N[2], prot_voxels.shape[1])
    annotations = annotation_voxels.reshape(N[0], N[1], N[2], annotation_voxels.shape[1])
    
    with open(f'{voxel_features_path}/{pdb_id.lower()}{chain_id.upper()}.npy', 'wb') as f:
        np.save(f, features)
    with open(f'{voxel_annotations_path}/{pdb_id.lower()}{chain_id.upper()}.npy', 'wb') as f:
        np.save(f, annotations)
    return True

In [ ]:
HOLO_VOXEL_FEATURES_PATH = '../data/holo-voxel-features'
HOLO_VOXEL_ANNOTATIONS_PATH = '../data/holo-voxel-annotations'

# generate voxels for the HOLOs
with open(f'../cryptobench/whole_dataset.json', 'r') as json_file:
    dataset = json.load(json_file)

for apo_structure, holo_structures in dataset.items():
    apo_pockets = set()    
    for holo_structure in holo_structures:
        
        pdb_id = holo_structure['holo_pdb_id']
        chain_id = holo_structure['holo_chain']
        apo_pocket = holo_structure['apo_pocket_selection']
        pocket = holo_structure['holo_pocket_selection']
        id = pdb_id.lower() + chain_id.upper()
        
        # it gets stuck on 5c30. Just skip it, I am too furious to deal with it now
        if apo_structure == '5c30':
            break

        if os.path.isfile(f'{HOLO_VOXEL_FEATURES_PATH}/{pdb_id.lower()}{chain_id.upper()}.npy') \
            and os.path.isfile(f'{HOLO_VOXEL_ANNOTATIONS_PATH}/{pdb_id.lower()}{chain_id.upper()}.npy'):
            break
        
        new_apo_residues = [residue.split('_')[1] for residue in apo_pocket if residue.split('_')[1] not in apo_pockets]
        # probably a homomer
        if (len(apo_pocket) - len(new_apo_residues)) / len(apo_pocket) > 0.75:
            continue
        apo_pockets.update(new_apo_residues)

        if not generate_voxels(pdb_id, chain_id, pocket, HOLO_VOXEL_FEATURES_PATH, HOLO_VOXEL_ANNOTATIONS_PATH):
            continue


### GENERATE VOXELS FOR APO STRUCTURES
This needs to be handled a bit differently. First, all the pockets in the APO need to be ensembled. Then the voxel generation can proceed. 

Furthermore, only the TEST subset is needed.

In [ ]:
APO_VOXEL_FEATURES_PATH = '../data/apo-voxel-features'
APO_VOXEL_ANNOTATIONS_PATH = '../data/apo-voxel-annotations'

# generate voxels for the HOLOs
with open(f'../cryptobench/test/test.json', 'r') as json_file:
    dataset = json.load(json_file)

for apo_pdb_id, holo_structures in dataset.items():
    apo_chain_id = holo_structures[0]['apo_chain']

    if os.path.isfile(f'{APO_VOXEL_FEATURES_PATH}/{apo_pdb_id.lower()}{apo_chain_id.upper()}.npy') \
        and os.path.isfile(f'{APO_VOXEL_ANNOTATIONS_PATH}/{apo_pdb_id.lower()}{apo_chain_id.upper()}.npy'):
        continue

    # collect all the pockets     
    apo_pockets = set()    
    for holo_structure in holo_structures:
        apo_pocket = holo_structure['apo_pocket_selection']
        apo_pockets.update(apo_pocket)
    
    if not generate_voxels(apo_pdb_id, apo_chain_id, list(apo_pockets), APO_VOXEL_FEATURES_PATH, APO_VOXEL_ANNOTATIONS_PATH):
        continue
    
